# Project Overview: Visualization for Toronto Ward Budgets (Year 2024 to Year 2033)

## Prerequisites
- **PowerBI**
- **Data Visualization**
- **Data Analysis**

## 1. Data Source and Description

### 1.1 Data Source
The data used in this project is **automatically** retrieved and downloaded by Python from various Excel files available at the following link. [Toronto Open Data](https://open.toronto.ca/dataset/budget-capital-budget-plan-by-ward-10-yr-approved/
). The downloaded data is stored in the data folder within the current year's work directory.

<img src="data files.png" alt="data files" width="400" height="400">


### 1.2 Data Description

Each file contains financial data for 44 (25 from 2019) wards in Toronto, covering the time span **from 2010 to 2033**. The columns detail annual budgets, and key columns of interest are:

- `Ward/Project Number`: Identifier for each ward and project.
- `Year` columns: Columns that represent different years' budget data.

To illustrate the budget allocation for Toronto over the next 10 years, we will analyze the latest 2024-2033 budget file.

## 2. Project Objective (Data Visualization)

The primary objective is to use Power BI to create an **interactive** plot that visualizes Toronto's budget planning for the next 10 years.

### 2.1 Chart Features
- **Vertical Comparison**: Compare the budget changes of the same ward across different years, showing how the budget evolves over time for that specific ward.
- **Horizontal Comparison**: Compare the budgets of different wards for the same year, highlighting how budgets vary among wards in a given year.

![Dynamic Map Chart](viz_by_powerbi.gif)

## 3. Data Preparing 

To improve efficiency, the data required for Power BI visualization will be automated and retrieved by Python.

### 3.1 Reading Excel Files

### 3.2 Extracting and Cleaning Data (ETL)

## 4. Data Calculating for Visualization 

Prepare the data for visualization by:

- Aggreating budgets for each ward.
- Adding new columns as needed.
- Filling missing values with fillna.
- Ensuring data accuracy for accurate visualizations.

<img src="viz_by_powerbi.png" alt="PowerBI" width="600" height="600">

## 5. Data Visualization (Map Chart)
   - Use PowerBI to create an interactive map chart.
   - Display annual budget plans for each ward, showing how budgets change both from year to year and across different wards.
   

(End)


### 0. Install Python Modules

In [122]:
import subprocess
import sys

# 0.1 The following function installs/tests the major required packages for this script

def install_and_import(package):
    try:
        import_name = package
        __import__(import_name)
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} installed successfully.")
    finally:
        globals()[package] = __import__(import_name)


import_packages = ['selenium', 'webdriver_manager', 'xlrd', 'openpyxl', 'xls2xlsx']

for package in import_packages:
    install_and_import(package)
print('Packages have been installed.')

Packages have been installed.


### 1. Import Libraries and Define Variables

In [123]:
# Get download links
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import re

# Download files
import requests
import os

# Data ETL
import pandas as pd
import openpyxl
from openpyxl import Workbook, load_workbook
from xls2xlsx import XLS2XLSX

# Data Integration
from geopy.geocoders import Nominatim

# Suppress specific warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='openpyxl')
warnings.simplefilter(action='ignore', category=FutureWarning)


source_data_link         = 'https://open.toronto.ca/dataset/budget-capital-budget-plan-by-ward-10-yr-approved/'
source_button_XPATH      = '//*[@id="header-Download"]/div/h3/button'
link_XPATH               = '//*[@id="table-resources"]'
download_folder          = 'data'
ward_column_list         = ['Ward', 'Ward/Project Number', 'Ward/Project Num.', 'Ward Name']
inserted_column          = 'Year'
# rename_column_1          = 'Budget_Year_1'
# rename_column_2          = 'Budget_Year_2'
extracted_data_file      = 'budget_data.csv'
report_data_file         = 'report_data.csv'


### 2. Data Preparing
 
- **2.1 Function to get download links for multiple Excel files**

**_Please note that this code will automatically use Google Chrome to search for download links. If you do not have the Google Chrome browser installed, please manually download the data from the source data link (in the previous cell) to the 'data' folder in your current working directory._**

In [124]:
def get_links(url, button_XPATH, links_area_XPATH):

    # Set up Selenium options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode (no browser window)

    # Initialize the WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Open the target URL
    driver.get(url)

    # Wait for the button to be clickable and click it
    time.sleep(1)  # Adjust wait time as needed
    button = driver.find_element(By.XPATH, button_XPATH)  # Adjust XPath as needed
    button.click()

    # Wait for the content to load
    time.sleep(1)  # Adjust wait time as needed

    # Find the specific area containing the links
    links_area = driver.find_element(By.XPATH, links_area_XPATH)

    # Find all links within the specified area
    links = links_area.find_elements(By.TAG_NAME, 'a')

    # Extract all download links (including .xls and .xlsx) that contain "download"
    download_links = [
        link.get_attribute('href') for link in links
        if 'download' in link.get_attribute('href').lower() and re.search(r'\.(xls|xlsx)$', link.get_attribute('href'), re.IGNORECASE)
    ]

    # Close the browser
    driver.quit()
    
    return download_links

- **2.2 Function to download Files using the download links**

In [125]:
def download_files(download_links, download_folder):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    for link in download_links:
        # Extract the file name from the link
        file_name = os.path.join(download_folder, link.split('/')[-1])

        # Send a GET request to download the file
        response = requests.get(link)
        response.raise_for_status()  # Ensure we notice bad responses

        # Write the file to the specified folder
        with open(file_name, 'wb') as file:
            file.write(response.content)

        print(f"Downloaded {file_name}")


- **2.3 Data ETL (Extraction, Transformation, Loading)**

- 2.3.1 Subfunction 1 to get the year column

In [126]:
def get_year_from_column(name):
    # Function to check if a column name can be converted to an int
    try:
        return int(name)
    except:        
        return None

- 2.3.2 Subfunction 2 to convert .xls to .xlsx

In [127]:
def convert_xls_to_xlsx(xls_path, xlsx_path):
    
    xls_file = XLS2XLSX(xls_path)
    xls_file.to_xlsx(xlsx_path)
    

- 2.3.3 Subfunction 3 to fill the unmerged cells

In [128]:
def fill_unmerged_cells(file_path):
    workbook = load_workbook(filename=file_path)

    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        for merge in list(sheet.merged_cells.ranges):
            min_col, min_row, max_col, max_row = merge.bounds
            top_left_cell_value = sheet.cell(row=min_row, column=min_col).value
            sheet.unmerge_cells(str(merge))
            for row in range(min_row, max_row + 1):
                for col in range(min_col, max_col + 1):
                    sheet.cell(row=row, column=col).value = top_left_cell_value
    workbook.save(file_path)
    print(f"Unmerged cells have been saved for Year: {(os.path.basename(file_path))[0:4]}")  
    

- 2.3.4 Subfunction 4 to convert string to numerical values

In [129]:
def to_numeric(value):
    try:
        return float(value)
    except ValueError:
        return 0

- 2.3.5 Subfunction 5 to process the data

In [130]:
def data_processing(file_path, ward_column_list, sheet_name=None):
    file_name = os.path.basename(file_path)
    year_number = file_name[0:4]
    filtered_data = None
    # 1) Read the Excel file and convert the data to string temporarily
    if sheet_name:
        try: 
            # Read a specific sheet into a DataFrame
            excel_data = pd.read_excel(file_path, header=None, sheet_name=sheet_name, dtype=str,engine='openpyxl')
        except: 
            excel_data = pd.read_excel(file_path, header=None, sheet_name=sheet_name, dtype=str,engine='xlrd')
        # Wrap DataFrame in a dictionary to handle uniformly
        excel_data = {sheet_name: excel_data}
    else:
        try:
            # Read all sheets into a dictionary
            excel_data = pd.read_excel(file_path, header=None, dtype=str, sheet_name=None, engine='openpyxl')
        except:
            excel_data = pd.read_excel(file_path, header=None, dtype=str, sheet_name=None, engine='xlrd')
    # 2) Iterate over the sheets and find the actual header
    for sheet_name, sheet_data in excel_data.items():
        # Flatten the DataFrame to a Series for easier searching
        # flat_series = sheet_data.stack().astype(str)
        
        ward_row_index = None
        # Check if 'ward_column' is found in any cell
        for ward_column in ward_column_list:
            # print(f'Searching for Ward_column : {ward_column}')
            # if flat_series.str.contains(ward_column, case=False, na=False).any():

            # Find all the index of the row containing 'Ward/Project Number' or 'Ward'
            matched_rows = sheet_data[sheet_data.apply(lambda row: row.astype(str).str.strip() == ward_column).any(axis=1)]
            if len(matched_rows) == 0:
                # The result contains nothing
                continue
            elif len(matched_rows) ==1 and int(year_number)<2019:
                ward_row_index = matched_rows.index[0]
            else:
                # Multiple values have been found                                  
                # ward_column =='Ward' 
                # print(matched_rows)
                for index, row in matched_rows.iterrows():
                    # print(f'Index is "{index}", Row is "{row}"')
                    # Check if the column to the right of 'Ward' contains 'Project Name'
                    ward_col_index = row[row.astype(str).str.strip() == ward_column].index[0]
                    # print(f"Ward_col_index is {ward_col_index}")
                    # if ward_col_index + 1 < len(sheet_data.columns):
                    #     # print(len(row))
                    #     # print(type(row))
                    project_col = row[row.astype(str).str.strip() == 'Project Name']
                    # print(f"Project_col is {project_col}")
                    if project_col.empty:
                        continue
                    else:
                        if int(year_number) >=2019:
                            ward_row_index = index
                        
                        elif int(year_number) == 2010:
                            project_col_index = project_col.index[0]
                            if  ward_col_index+1 == project_col_index :
                                ward_column = row[ward_col_index]
                                # 'Ward' is a desired column name
                                ward_row_index = index
                            break
                 
            if ward_row_index >=0:
                # Ward Column is found    
                print(f"'{ward_column}' found in the sheet '{sheet_name}'.")
                above_budget_cell_value = None
                row_values = sheet_data.iloc[ward_row_index].tolist()
               
                if 'Budget' in row_values:
                    # Some files contain the current year number above the "Budget" cell
                    budget_cell_index = row_values.index('Budget')

                    # Ensure the index is not None:
                    if budget_cell_index > 0:
                        above_budget_cell_value = sheet_data.iloc[ward_row_index - 1, budget_cell_index]
                        
                # 4) Remove all rows above the 'Ward/Project Number' row and reset the index
                if ward_row_index >0 :
                    data = sheet_data.iloc[ward_row_index:].reset_index(drop=True)
                else:
                    data = sheet_data.reset_index(drop=True)
                # 5) Set the first row of data as column headers
                new_header = data.iloc[0]
                data = data[1:]
                data.columns = new_header
                if above_budget_cell_value:
                    data.rename(columns={'Budget': above_budget_cell_value}, inplace=True)
                
                # 6) Extract all column names and find columns that can be converted to year
                year_columns = [(col, get_year_from_column(col)) for col in data.columns]
                year_columns = [col for col, year in year_columns if year is not None]
                
                # 7) Get all year columns
                # Note: copy() is used to prevent year columns from changes
                selected_columns = year_columns.copy()
                # 8) Keep only necessary columns
                if int(year_number)>=2019:
                    selected_columns.extend([ward_column, 'Ward Number'])
                else:
                    selected_columns.append(ward_column)
                selected_data = data[selected_columns]
                    
                # 9) Filter the data 
                selected_data.loc[:,ward_column] = selected_data[ward_column].fillna(' ')
                # print(selected_data['Ward Number'].unique())
                # print(year_number)
                if year_number == '2010':
                    filtered_data = selected_data[selected_data[ward_column].str.startswith('Ward')]
                    # print(f'filtered_data is {filtered_data}')
                    filtered_data = filtered_data.dropna(subset=selected_columns)
                elif int(year_number) >=2019:
                    # As Ward Number has been chosen, those indicating non-numerical value will be removed
                    selected_data = selected_data[selected_data['Ward Number'].apply(lambda x: str(x).isdigit())]
                    # print(selected_data.head(5))
                    original_data= selected_data.copy()
                    
                    # print(original_data['Ward Number'].unique())
                    for column in year_columns:
                        # print(column)
                        original_data[column] = original_data[column].apply(to_numeric)
                    # print(original_data['Ward Number'].unique())
                    # print(original_data)
                    # List of columns to sum
                    columns_to_sum = year_columns  
                    # Perform the sum operation and keep all other columns
                    original_data[columns_to_sum] = original_data.groupby('Ward Number')[columns_to_sum].transform('sum')
                    filtered_data = original_data
                    # filtered_data= original_data.groupby('Ward Number').sum().reset_index()\
                    # Try to avoid using groupby this way, the non-numerical columns will be grouped as well
                    filtered_data = filtered_data.rename(columns={
                        'Ward Number':'Ward_Number', 
                        ward_column:ward_column_list[0],
                    })
                    # print(filtered_data)
                else:
                    filtered_data = selected_data[selected_data[ward_column].str.endswith('Total')]

                # 10) Drop duplicates and null values
                filtered_data = filtered_data.drop_duplicates()
                filtered_data = filtered_data.dropna()
    # print(filtered_data)    
    return filtered_data

- 2.3.6 Function for Data ETL

In [131]:
def data_etl(download_folder, ward_column_list, sheet_name=None):    
    

    # 1) Get the absolute path of the download folder
    folder_path = os.path.abspath(download_folder)

    # 2) Iterate over all Excel files in the folder to look for the data file
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
            year_numbers = file_name[0:9]
            if '2024-2033-' in file_name:
                
                file_path = os.path.join(folder_path, file_name)
                if file_name.endswith('.xls') :
                    # !This is very important! The merged cells in the Excel must be unmerged and filled before loading the values into a dataframe
                    xlsx_path = file_path.replace('.xls','.xlsx')
                    # print(file_name)
                    convert_xls_to_xlsx(file_path,xlsx_path)
                    fill_unmerged_cells(xlsx_path)
                    continue
                
                print(f'Extracting data for Year "{year_numbers}"....')
                sheet_names = pd.ExcelFile(file_path).sheet_names
                filtered_data = data_processing(file_path, ward_column_list, sheet_names[0])
    
    # Unpivot
    filtered_data = pd.melt(filtered_data, id_vars=[ward_column_list[0], 'Ward_Number'], var_name='Year', value_name='Budget')
    # print(filtered_data)
    return filtered_data
            

### 3. Main() Function to streamline and automate data visualization, data processing and data analysis.

In [132]:
def main():

    # 1. Get the download links from data source
    download_links = get_links(source_data_link, source_button_XPATH, link_XPATH)

    # Print all download links
    # print("Download links:")
    # for link in download_links:
    #     print(link)
    print("Step 1. Download links have been obtained.")

    # 2. Download files using the download links
    # Only download the data for this project
    for link in download_links:
        if '2024-2033-' in link:
            download_links = [link]
            download_files(download_links, download_folder)

    print(f"Step 2. Data files have been download in '{os.path.abspath(download_folder)}'. ")
    
    # 3. Data Process (ETL)
    budget_data = data_etl(download_folder, ward_column_list, None)
    if budget_data.empty:
        return f"Budget_data not found in the {download_folder} folder."
    # print(budget_data.head(5))
    
    # Save the extracted data into .csv file
    budget_data.to_csv(extracted_data_file)
    print("Step 3. Budget data has been extracted and saved.")

    # # 4. Data Calculation for Reporting(Visualization)
    # report_data = data_integrating(extracted_data_file,ward_column_list)
    # report_data.to_csv(report_data_file)
    # print('Step 4. Report data have been calculated and saved.')
    # # print(report_data.head(5))
    
    
if __name__ == "__main__":
    main()

Step 1. Download links have been obtained.
Downloaded data/2024-2033-capital-budget-and-plan-details.xlsx
Step 2. Data files have been download in '/Users/julia/Desktop/Life/DSI/9. visualization/04_cohort_three/live_code/data'. 
Extracting data for Year "2024-2033"....
'Ward' found in the sheet '2024-2032'.
Step 3. Budget data has been extracted and saved.
